# Real-World Crime Analytics for the London Metropolitan Area

## 1. Environment Setup (Linux)

### 1.1. Update Linux Packages

In [ ]:
!sudo apt update

### 1.2. Install Python and Packages (pip & venv)

In [ ]:
!sudo apt install -y python3 python3-pip python3-venv

### 1.3. Create a Virtual Environment

In [ ]:
!python3 -m venv venv

### 1.4. Activate the Virtual Environment

In [ ]:
!source venv/bin/activate

### 1.5. Install the Required Python Packages

In [ ]:
%pip install -r requirements.txt

## 2. Data Ingestion

### 2.1. Create Folders to Store Data

In [ ]:
import os

# Create datasets folder if it doesn't exist
os.makedirs("Datasets", exist_ok=True)
os.makedirs("Datasets/Raw-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Map-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Police-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Income-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Population-Data", exist_ok=True)

### 2.2. Download Data

#### 2.2.1. Download Income Data - <a>www.ons.gov.uk</a>

In [ ]:
!wget -O Datasets/Raw-Data/Income-Data/Income-MSOA.xlsx "https://www.ons.gov.uk/file?uri=/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/smallareaincomeestimatesformiddlelayersuperoutputareasenglandandwales/financialyearending2020/saiefy1920finalqaddownload280923.xlsx"
print("Downloaded to Datasets/Raw-Data/Income-Data/Income-MSOA.xlsx")

#### 2.2.2. Download Population Data - <a>www.ons.gov.uk</a>

In [ ]:
!wget -O Datasets/Raw-Data/Population-Data/Population-LSOA.xlsx "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/lowersuperoutputareamidyearpopulationestimates/mid2022revisednov2025tomid2024/sapelsoasyoa20222024.xlsx"
print("Downloaded to Datasets/Raw-Data/Population-Data/Population-LSOA.xlsx")

#### 2.2.3. Download Police Data - <a>data.police.uk</a>

In [ ]:
!wget -O Datasets/Raw-Data/Police-Data/RAW-POLICE-2022-10-TO-2025-09.zip "https://data.police.uk/data/archive/2025-09.zip"
print("Downloaded to Datasets/Raw-Data/RAW-POLICE-2022-10-TO-2025-09.zip")

In [ ]:
# Extract the police data zip file
!unzip -o Datasets/Raw-Data/RAW-POLICE-2022-10-TO-2025-09.zip -d Datasets/Raw-Data/Police-Data
print("Extracted police data to Datasets/Raw-Data/Police-Data/")

In [ ]:
# Clean up the zip file to save space
!rm -rf Datasets/Raw-Data/RAW-POLICE-2022-10-TO-2025-09.zip

### 2.2.4. Download Map Data - <a>geoportal.statistics.gov.uk</a>

#### Note: Map Data from ONS Geography Can't be Downloaded Directly, So Use the Below Links to Download Them.

1. LSOAs - <a>https://geoportal.statistics.gov.uk/datasets/ons::output-areas-december-2021-boundaries-ew-bgc-v2/about</a>

2. MSOAs - <a>https://geoportal.statistics.gov.uk/datasets/ons::middle-layer-super-output-areas-december-2021-boundaries-ew-bfc-v7-2/about</a>

## 3. Data Cleaning

### 3.1. Convert XLSX (Excel) files to CSV using pandas

In [ ]:
import pandas as pd
import os
import re

# Safe filename from sheet name
def sanitize(name: str) -> str:
    # Replace non-alphanumeric with underscores, strip, collapse repeats
    name = re.sub(r"[^A-Za-z0-9]+", "_", name).strip("_")
    return re.sub(r"_+", "_", name) or "Sheet"

# Convert Income-MSOA.xlsx: write one CSV per sheet
income_xlsx_path = "Datasets/Raw-Data/Income-Data/Income-MSOA.xlsx"
income_out_dir = "Datasets/Raw-Data/Income-Data/Income-MSOA"
os.makedirs(income_out_dir, exist_ok=True)

try:
    xls_income = pd.ExcelFile(income_xlsx_path)
    for sheet in xls_income.sheet_names:
        df = pd.read_excel(xls_income, sheet_name=sheet)
        safe = sanitize(sheet)
        out_path = os.path.join(income_out_dir, f"Income-MSOA-{safe}.csv")
        df.to_csv(out_path, index=False)
    print(f"Exported {len(xls_income.sheet_names)} sheets from {income_xlsx_path} to {income_out_dir}")
except Exception as e:
    print("Failed to process income workbook:", e)

# Convert Population-LSOA.xlsx: write one CSV per sheet
population_xlsx_path = "Datasets/Raw-Data/Population-Data/Population-LSOA.xlsx"
population_out_dir = "Datasets/Raw-Data/Population-Data/Population-LSOA"
os.makedirs(population_out_dir, exist_ok=True)

try:
    xls_pop = pd.ExcelFile(population_xlsx_path)
    for sheet in xls_pop.sheet_names:
        df = pd.read_excel(xls_pop, sheet_name=sheet)
        safe = sanitize(sheet)
        out_path = os.path.join(population_out_dir, f"Population-LSOA-{safe}.csv")
        df.to_csv(out_path, index=False)
    print(f"Exported {len(xls_pop.sheet_names)} sheets from {population_xlsx_path} to {population_out_dir}")
except Exception as e:
    print("Failed to process population workbook:", e)

Exported 8 sheets from Datasets/Raw-Data/Income-Data/Income-MSOA.xlsx to Datasets/Raw-Data/Income-Data/Income-MSOA
Exported 7 sheets from Datasets/Raw-Data/Population-Data/Population-LSOA.xlsx to Datasets/Raw-Data/Population-Data/Population-LSOA
Exported 7 sheets from Datasets/Raw-Data/Population-Data/Population-LSOA.xlsx to Datasets/Raw-Data/Population-Data/Population-LSOA


In [ ]:
# Import and initialize Spark
import pyspark
from pyspark.sql import SparkSession  
from pyspark.sql import DataFrame

spark = SparkSession.builder \
    .appName("Real-World Crime Analytics for the London Metropolitan Area") \
    .getOrCreate()

print("Spark version:", spark.version)

## 4. Data Transformation

## 5. Data Querying